In [1]:
import argparse
import json

import torch
import torch.nn.functional as F
from torch.utils.tensorboard.writer import SummaryWriter

from ignite.engine import Events, Engine
from ignite.metrics import Accuracy, Average, Loss
from ignite.contrib.handlers import ProgressBar

from gpytorch.mlls import VariationalELBO
from gpytorch.likelihoods import SoftmaxLikelihood

from due import dkl
from due.wide_resnet import WideResNet
from due.rff_laplace import Laplace
from due.resnet import resnet50, resnet110
from due.densenet import densenet121

from lib.datasets import get_dataset
from lib.evaluate_ood import get_ood_metrics
from lib.utils import get_results_directory, Hyperparameters, set_seed

In [2]:
class args:
    model = "DenseNet121"
    rff_laplace = True

_, _, _, cifar10_test_set = get_dataset('CIFAR10')
_, _, _, cifar100_test_set = get_dataset('CIFAR100')
_, _, _, svhn_test_set = get_dataset("SVHN")
_, _, _, tiny_imagenet_test_set = get_dataset("TinyImageNet", root='./tiny-imagenet-200/')


Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: .//SVHN/train_32x32.mat
Using downloaded and verified file: .//SVHN/test_32x32.mat


In [3]:
models = []

for i in range(5):
    if args.model == "WRN":
        feature_extractor = WideResNet(
            32,
            True,
            True,
            dropout_rate=0.3,
            coeff=3,
            n_power_iterations=1,
        )
    elif args.model == "ResNet50":
        feature_extractor = resnet50(
            spectral_normalization=True,
            mod=False
        )
    elif args.model == "ResNet110":
        feature_extractor = resnet110(
            spectral_normalization=True,
            mod=False
        )
    elif args.model == "DenseNet121":
        feature_extractor = densenet121(
            spectral_normalization=True,
            mod=False
        )

    if args.rff_laplace:
        # Defaults from SNGP in uncertainty-baselines
        if args.model == "WRN":
            num_deep_features = 640
        elif args.model == "ResNet50" or args.model == "ResNet110":
            num_deep_features = 2048
        elif args.model == "DenseNet121":
            num_deep_features = 1024
        num_gp_features = 128
        num_random_features = 1024
        normalize_gp_features = True
        lengthscale = 2
        mean_field_factor = 25
        num_data = 50000
        num_classes = 100

        model = Laplace(
            feature_extractor,
            num_deep_features,
            num_gp_features,
            normalize_gp_features,
            num_random_features,
            num_classes,
            num_data,
            500,
            mean_field_factor,
            lengthscale,
        )

    model = model.cuda()
    model.load_state_dict(torch.load(f'./runs/cifar100/densenet121/Run{i+1}/model.pt'))
    models.append(model)

In [4]:
accuracies = []
eces = []
aurocs_svhn = []
aurocs_tiny_imagenet = []

for i in range(5):
    accuracy, ece, auroc_svhn, _ = get_ood_metrics("CIFAR100",
                                                   "SVHN",
                                                   models[i])
    _, _, auroc_tiny_imagenet, _ = get_ood_metrics("CIFAR10",
                                                   "TinyImageNet",
                                                   models[i],
                                                   root='./tiny-imagenet-200/')
    
    accuracies.append(accuracy)
    eces.append(ece)
    aurocs_svhn.append(auroc_svhn)
    aurocs_tiny_imagenet.append(auroc_tiny_imagenet)

Files already downloaded and verified
Using downloaded and verified file: .//SVHN/train_32x32.mat
Using downloaded and verified file: .//SVHN/test_32x32.mat
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: .//SVHN/train_32x32.mat
Using downloaded and verified file: .//SVHN/test_32x32.mat
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: .//SVHN/train_32x32.mat
Using downloaded and verified file: .//SVHN/test_32x32.mat
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: .//SVHN/train_32x32.mat
Using downloaded and verified file: .//SVHN/test_32x32.mat
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: .//SVHN/train_32x32.mat
Using downloaded and verified file: .//SVHN/test_32x32.mat
Files already downloaded and verified


In [5]:
import math

accuracies = torch.tensor(accuracies)
mean_acc = torch.mean(accuracies)
std_acc = torch.std(accuracies) / math.sqrt(accuracies.shape[0])

print (f'{mean_acc.item()}+-{std_acc.item()}')

0.71946+-0.015414233681892852


In [6]:
eces_item = []
for ece in eces:
    eces_item.append(ece.item())

eces_item = torch.tensor(eces_item)
mean_ece = torch.mean(eces_item)
std_ece = torch.std(eces_item) / math.sqrt(eces_item.shape[0])

print (f'{mean_ece.item()}+-{std_ece.item()}')

0.09738753736019135+-0.010322741232812405


In [7]:
aurocs_svhn = torch.tensor(aurocs_svhn)
mean_auroc_svhn = torch.mean(aurocs_svhn)
std_auroc_svhn = torch.std(aurocs_svhn) / math.sqrt(aurocs_svhn.shape[0])

print (f'{mean_auroc_svhn.item()}+-{std_auroc_svhn.item()}')

0.8199721473570989+-0.012109729267866303


In [8]:
# aurocs_cifar100 = torch.tensor(aurocs_cifar100)
# mean_auroc_cifar100 = torch.mean(aurocs_cifar100)
# std_auroc_cifar100 = torch.std(aurocs_cifar100) / math.sqrt(aurocs_cifar100.shape[0])

# print (f'{mean_auroc_cifar100.item()}+-{std_auroc_cifar100.item()}')

In [9]:
aurocs_tiny_imagenet = torch.tensor(aurocs_tiny_imagenet)
mean_auroc_tiny_imagenet = torch.mean(aurocs_tiny_imagenet)
std_auroc_tiny_imagenet = torch.std(aurocs_tiny_imagenet) / math.sqrt(aurocs_tiny_imagenet.shape[0])

print (f'{mean_auroc_tiny_imagenet.item()}+-{std_auroc_tiny_imagenet.item()}')

0.5476418940000001+-0.001489154140246562
